# P2 - Statistics + Visualization


This project is intended to give you experience with statistics and working with a large data set.





## Project Setup

You should use the following modules in this assignment and not need any additional modules. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import datetime
import os 
import re
import platform
import sys
import importlib
from packaging.version import Version, parse 

if os.environ["HOME"]=='/home/jovyan':
    !pip install --upgrade otter-grader
    
import otter
grader = otter.Notebook()

# Flight Delays

Suppose the Houghton County Airport (CMX) is ready to renew its passenger airplane contract.  Airport officials and interested passengers would like to select an airline and connecting city that has reliable service.  *Currently, CMX is being served by United Airlines through Chicago-O'hare (ORD); it has been served by Delta in the past with connections to Minneapolis-St. Paul (MSP).* 

In order to make an informed decision, you have been tasked to look at flight delay statistics for two potential connector airports: 

* Chicago - O'hare (ORD) 
* Detroit (DTW) 

The data you are provided comes from the US Department of Transportation's [Bureau of Transportation Statistics (BTS)](https://www.bts.gov).  In particular, it comes from the Reporting Carrier On-Time Performance data tables:  
https://www.transtats.bts.gov/TableInfo.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr&V0s1_b0yB=D

Information on the variables can be found in the Field Information:  
https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ


The data is structured with the following elements: 

* "YEAR"
* "MONTH"
* "DAY_OF_MONTH"
* "OP_UNIQUE_CARRIER" - Unique Carrier Code
* "TAIL_NUM" - Tail number 
* "OP_CARRIER_FL_NUM" - Flight number 
* "ORIGIN" - Origin Airport code 
* "DEST" - Destination Airport code 
* "CRS_DEP_TIME" - Computer Reservation System Departure Time (local time: hhmm)
* "DEP_TIME" - Actual Departure Time (local time: hhmm)
* "DEP_NEXT_DAY" - the flight departed on the: (scheduled day = 0), (next day = 1), (following day = 2), or (day before  = -1)
* "CRS_ARR_TIME" - Computer Reservation System Arrival Time (local time: hhmm)
* "ARR_TIME" - Actual Arrival Time 
* "ARR_NEXT_DAY" - the flight arrived on the: (scheduled day = 0), (next day = 1), (following day = 2), or (day before  = -1)
* "CANCELLED" - Cancelled Flight Indicator (1 = Yes)
* "DIVERTED" - Diverted Flight Indicator (1 = Yes)

You have access to 2 years of data Jan 2022 - Dec 2023. 

The data has also been restricted to several major carriers and their regional services: 

| Airline Code | Airline Name | Regional Partner | 
|--------------|--------------|------------------|
| UA      | United Airlines   | NA  | 
| DL      | Delta Airlines    | NA  |
| OO      | Skywest Airlines  | United  | 
| 9E      | Endeavor Air      | Delta     | 


## Access the Data 

The data is available on [Kaggle](https://www.kaggle.com/datasets/lebrown/un5550-fa24-p2/data)

Once you download the data make sure it is saved in your project directory: 

```
class-folder/
    p2/
        flight-final.csv
        img/
        p2.ipynb
```

<!-- BEGIN QUESTION -->

## Q0 - Setup 

The following code checks to make sure your notebook is running on Gradescope, the cloud, or linux lab machines. 

While you may develop on your own machines or places like Colab or Github, you should run your final submission on the campus Linux machines using the `un5550fa24` environment. 


_Type your answer here, replacing this text._

In [ ]:
# DO NOT CHANGE THIS CODE

# flag if notebook is running on Gradescope 
if re.search(r'amzn', platform.uname().release): 
    GS = True
else: 
    GS = False

# flag if notebook is running on Colaboratory 
try:
  import google.colab
  COLAB = True
except:
  COLAB = False

# flag if running on Linux lab machines. 
cname = platform.uname().node
if re.search(r'(guardian|colossus|c28|coc-15954-m)', cname):
    LLM = True 
else: 
    LLM = False

print("System: GS - %s, COLAB - %s, LLM - %s" % (GS, COLAB, LLM))

In [ ]:
# DO NOT CHANGE THIS CODE
pver = sys.version 
print(pver) 

In [ ]:
# DO NOT CHANGE THIS CODE
env2 =!conda info | grep 'active env'
print(env2)

In [ ]:
# DO NOT CHANGE THIS CODE
OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "x1b[41m[FAIL]\x1b[0m"

def import_version(pkg, req_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        ver = mod.__version__
        if Version(ver) != req_ver:
            print(FAIL, "%s version %s required, but %s installed."
                  % (lib, req_ver, ver))
        else:
            print(OK, '%s version %s' % (pkg, ver))
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return (mod, Version(ver), req_ver)

requirements = {'numpy': parse("2.1.0"), 'scipy': parse("1.14.1"),
                'matplotlib': parse("3.9.2"), 'pandas': parse("2.2.2"),
                'otter': parse("6.0.4"), 'seaborn': parse('0.13.2'),
                'dill': parse('0.3.7'), 'sklearn': parse("1.5.1")
                }

pks = []
for lib, required_version in list(requirements.items()):
    pks.append(import_version(lib, required_version))

<!-- END QUESTION -->

## Q1 - Load Data 

Read in the data.    

Set the column names to be: `year`, `month`, `day`, `carrier`, `tailNum`, `flNum`, `origin`, `dest`, `crsDepTime`, `actDepTime`, `depNextDay`, `crsArrTime`, `actArrTime`, `arrNextDay`, `cancelled`, `diverted` 
 
You do not need to do any special preprocessing or changing of the data types, we will be during specific processing of the data in the coming questions. 

In [ ]:
# Read in data, set DataFrame columns to names above 

flights = ...


flights.head()

In [ ]:
grader.check("q1")

## Aside : Style 

In the following questions you are going to be chaining together many expressions, perhaps creating code that is longer than 79 characters long (the maximum suggested by the [PEP-8 style guides](https://peps.python.org/pep-0008/#maximum-line-length)). 

    The preferred way of wrapping long lines is by using Python’s implied line 
    continuation inside parentheses, brackets and braces. Long lines can be broken 
    over multiple lines by wrapping expressions in parentheses. These should be 
    used in preference to using a backslash for line continuation.
    
Using parenthesis surrounding code make it possible to break the code into multiple lines for readability. 

Here are some examples: 

In [ ]:
temp = (flights[(flights['dest'] == 'CMX') | 
                (flights['origin'] == 'CMX')])

In [ ]:
temp = (
    flights[(flights['dest'] == 'CMX') | 
            (flights['origin'] == 'CMX')]
)

In [ ]:
temp = (flights
        .groupby('month')['diverted']
        .sum())

In [ ]:
temp = (
    flights
    .groupby('month')['diverted']
    .sum()
)

## Q2 - Explore Data

Let's start to explore the data.  

In [ ]:
print("Flight data: %8d rows,  %d columns" % (flights.shape[0], flights.shape[1]))

We want to remove flights that were cancelled or diverted from the rest of our analysis. 

But, first let's examine cancelled or diverted flights. 



### Q2a - Diverted and Cancelled Flights 

You will need to determine the number of diverted flights, `num_divert` and the number of cancelled flights, `num_cancel`. 

Next, determine the destination airport(s), by their code, with the highest proportion of diverted and cancelled flights `airport_highest_prop_divert` and `airport_highest_prop_cancel`.

In [ ]:
# num_divert - number of flights that were diverted. 
# num_cancel - number of flights that were cancelled 
num_divert = ...
num_cancel = ...
print('Num. of diverted flights:  %7d\nNum. of cancelled flights: %7d\n' % \
      (num_divert, num_cancel))

In [ ]:

airport_high_prop_divert = ...

airport_high_prop_cancel = ...


print('Airport with highest proportion of flights diverted:  %s' % airport_highest_prop_divert)
print('Airport with highest proportion of flights cancelled: %s' % airport_highest_prop_cancel)

In [ ]:
grader.check("q2a")

<br> 

Do these values make sense?  You can lookup airport codes on Google. 

<br>

**Note** the local airport in Houghton / Hancock is `CMX`.

### Q2b - Filter Cancelled and Diverted Flights

Now, let's remove the cancelled and diverted flights from further analysis. 

After removing both flights, reset the index on the remaining flights in the `flights` DataFrame.

In [ ]:
# Remove the cancelled and diverted flights.  
# Then, reset the index of the flights DataFrame
flights = ...
...
print("Flight data:   %8d rows" % flights.shape[0])

In [ ]:
grader.check("q2b")

### Q2c - Flights at `DTW` and `ORD`

Report the number of flights departing and arriving from each of the two airports under study: `DTW` and `ORD`. 

In [ ]:
# numArrDTW - number of flights arriving at DTW 
# numDepDTW - number of flights departing DTW
numArrDTW = ...
numDepDTW = ...
print("DTW flights:   %7d arrivals,  %7d departures" % (numArrDTW, numDepDTW))

In [ ]:
# numArrORD - number of flights arriving at ORD
# numDepORD - number of flights departing ORD
numArrORD = ...
numDepORD = ...
print("ORD flights:   %7d arrivals,  %7d departures" % (numArrORD, numDepORD))

*Note*: The number of flights arriving and departing from the two airports exceeds the total number of flights, because flights between the 2 airports are counted twice. 

In [ ]:
grader.check("q2c")

## Q3 - Time Information and Flight Delays. 

Both the departure and arrival times were read in as integers or floating-point numbers in local time format: `hhmm`. 

### Question 3a - Extract Hour and Minutes

Write two functions, `extract_hour` and `extract_mins` that converts the local time to hours and minutes, respectively. 

*Hint:* You may want to use modular arithmetic and integer division.

Notes: 

* The function should work on a single value passed or a Series passed in as an argument. 
* The function should not have a `for` loop, if it does you will lose points! 

In [ ]:
def extract_hour(time):
    """
    Extract hour information from the time given in hhmm format.

    Input:
        time (float64 or int64): Series of time given in hhmm format.  
          Takes on values in 0.0-2400.0 in float64 representation or 
          values in 0-2400 in int64 representation
 
    Returns:
        array (float64 or int64): Series of same dimension as input of hours  
          values should be 0-24, i.e., 24-hour day format
    
    Example: 1303 should return 13
    >>> extract_hour(1303.0)
    13
    """
    hours = ... 
    
    return hours

In [ ]:
def extract_min(time):
    """
    Extract minute information from the time given in hhmm time.

    Input:
        time (float64 or int64): Series of time given in hhmm format.  
          Takes on values in 0.0-2400.0 in float64 representation or 
          values in 0-2400 in int64 representation
    
    Returns:
        array (float64 or int64): Series of same dimension as input of minutes.  
          values should be 0-59
    
    Example: 1303 should return 3
    >>> extract_mins(1303.0)
    3
    """
    mins = ...
    return mins

Here are some examples of usage: 

```
>>> extract_hour(1450)
>>> extract_hour(flights['actDepTime'][100:200])
>>> extract_hour(flights['crsDepTime'])
>>> extract_min(1450)
>>> extract_min(flights['actDepTime'][100:200])
>>> extract_min(flights['crsDepTime'])
```

In [ ]:
grader.check("q3a")

### Q3b - Calculate Time in Minutes

Convert a time formatted in `hhmm` time in the number of minutes.    

For example, the following times in `hhmm` format converted to minutes are: `1005` is 605 minutes and `837` is 517 minutes and `1524` is 924 minutes. 

This function `convert_to_minutes` function, should make use of the two functions you wrote above: `extract_hour` and `extract_min`.

Comment: 

* The function should work on a single value passed or a Series passed in as an argument. 
* The function should not have a `for` loop, if it does you will lose points! 
* The function should use the functions `extract_hour` and `extract_min`.

In [ ]:
def convert_to_minutes(time):
    """
    Converts time in hhmm format to number of minutes
    
    Input:
       time (float64 or int64): Series of time given in hhmm format.  
          Takes on values in 0.0-2400.0 in float64 representation or 
          values in 0-2400 in int64 representation
    
    Returns:
        array (float64 or int64): Series of same dimension as input with 
           total mins
    
     
    Example: 1303 is converted to 783
    >>> convert_to_minutes(1303.0)
    783.0
    """
    # make use of the functions you wrote above: extract_hour, extract_min
    minutes = ... 
    return minutes

In [ ]:
grader.check("q3b")

### Q3c - Calculate Delayed Flights

You will add two new columns to the `flights` DataFrame that will contain the departure delay `depDelay` and arrival delay `arrDelay`.

To help answer this question, implement the helper functions, `calc_time_diff`.  Make use of the functions above, e.g., `convert_to_minutes`.


*Be careful for handling flights that that did not leave on their scheduled day indicated in the `depNextDay` and `arrNextDay` columns.*  
These two variables are encoded as: 
* 0, left/arrived on scheduled day 
* -1, left/arrived on the day before scheduled day
* 1, left/arrived 1 day after scheduled day 
* 2, left/arrived 2 days after scheduled day


Note, you can have *negative* flight delays for flights that leave / arrive early. 

Notes: 

* The function should work on Series passed in as an argument. 
* The function should not have a `for` loop, if it does you will lose points! 

In [ ]:
def calc_time_diff(x, y, nextDay):
    """ 
    Calculate the time difference, y - x, accounting for nextDay changes.

    Input:
        x,y (float64 or int64): Series of scheduled time given in hhmm format.  
          Takes on values in 0.0-2400.0 due to float64 representation or 
          values in 0-2400 in int64 representation
        nextDay (int): Series of next day indicator, takes on values: -1, 0, 1, 2
        
    Returns:
        array (float64): Series of input dimension with delay time
        
        
    Example: 1303 is converted to 783
             1305 is converted to 785
    >>> calc_time_diff(pd.Series([1303]), pd.Series([1305]), pd.Series([0]))
    2

    Example: 2320.0 is converted to 1400.0
               37.0 is converted to    37.0 
    >>> calc_time_diff(pd.Series([2320.0]), pd.Series([37.0]), pd.Series([1]))
    77.0
    """
    # make use of the convert_to_minutes function 
    
    diff = ...
    
    return diff

In [ ]:
flights['depDelay'] = ...
flights['arrDelay'] = ...

In [ ]:
grader.check("q3c")

## Q4 - Patterns of Delays 

Next, you will examine if there are any patterns in the delays.  


### Q4a - Delay Statistics

Present the mean, standard deviation, and median for departure and arrival delays for each airport, `DTW` and `ORD` as a destination in a DataFrame. 

Repeat the results where each airport, `DTW` and `ORD`, is the origin. 

**NOTE: all the meaningful autograded tests are hidden for this problem.** 

**Hint** You should consider using `groupby`

The DataFrame you create should look something like (note the values are not correct): 

<img src="img/q4a_sample.png" width="60%">

In [ ]:
# Calculate the mean, std, and median Depature Delay for flights leaving DTW/ORD
# Calculate the mean, std, and median Delay for flights arriving at DTW/ORD
# Report in requested DataFrames

destDelays = ... 

originDelays = ...

print(destDelays)
print(originDelays)

In [ ]:
grader.check("q4a")

<!-- BEGIN QUESTION -->

### Q4b - Distribution of Departure Delays 

Let's look at the distribution of departure delays for flights leaving `DTW` and `ORD`. 

Create overlapping density plots to examine these distributions. Only show the delays from -25 to 50 minutes. 

**Hint** Try using seaborns `kdeplot`

In [ ]:
 
...


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Q4c - Departure Delays by Day of Week

Let's examine if the departure delays differ by which day a flight is scheduled to leave.

Create a bar chart showing the mean departure delay for each day of the week. Report mean delays for Sun., Mon., Tues., ...

In order to get the day of the week, create a two new columns `dateVal` that is a datetime object of the date information of the flight.  Also, create a column `dayOfWeek` that encodes the day of the week for the flight.  You can make use of functions to convert datetime to day of the week.

We are looking at departure delays for flights leaving from ORD.


In [ ]:
# Look at departure delays for flights leaving ORD 
# Create a new column "dateVal" with a datetime object that may be useful
#  for this questions.  Also, add a column for "dayOfWeek"

# Create a bar chart of the mean `depDelay` by day of the week: Mon., Tues., ...

 
...


In [ ]:
grader.check("q4c")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Q4d - Dept. Delays by Month

Next, let's examine if the depature delays differ by month of the year. 

Create a bar chart showing the mean departure delay for each month of the year (Jan, Feb, Mar, Apr, ...) 

We again are looking at departure delays for flights leaving from ORD.

In [ ]:
# Create a bar plot with the mean dep. delay for each month. 
plt.figure(figsize=(8,6))

...


<!-- END QUESTION -->

### Q4e - Most Delayed Routes 

Let's examine the routes that are most delayed. 

Report out a DataSeries, `most_delay_routeA`,  with the airport as it's index and the value the mean arrival delay, only consider flights departing from `ORD` and routes that have at least 200 flights. 

Sort the DataSeries in descending order. 


Repeat the analysis for `DTW`, but use departure delay, report results in `most_delay_routeB` DataSeries.


In [ ]:
# Create a line chart plotting the mean dep. delay over time for each month 
#  of the 6 years 

...


In [ ]:
most_delay_routeA

In [ ]:
most_delay_routeB

In [ ]:
delay_routes.loc[(delay_routes[('depDelay', 'count')] > 200)].shape

In [ ]:
grader.check("q4e")

### Q4f - Delays by Carrier 

Because `ORD` is a United Hub and `DTW` is a Delta Hub we expect to see the delays grouped by airport may also be similar to the delays grouped by carrier.  Let's examine this. 

Consider all flights, not just those arriving or departing from `ORD` and `DTW`.  Report out the mean, standard deviation, and median arrival and departure delays for each carrier in a DataFrame. 

Label the carriers with their name, not code. 

| Airline Code | Airline Name | Regional Partner | 
|--------------|--------------|------------------|
| UA      | United Airlines   | NA  | 
| DL      | Delta Airlines    | NA  |
| OO      | Skywest Airlines  | United  | 
| 9E      | Endeavor Air      | Delta     | 

The DataFrame `carrierDelays` has columns: `carrier`, `arrDelayMean`, `arrDelayStd`, `arrDelayMedian`,
`depDelayMean`, `depDelayStd`, `depDelayMedian`

Sort the DataFrame by increasing mean arrival  delays.

In [ ]:

carrierDelays = ... 


carrierDelays

In [ ]:
grader.check("q4f")

## Bonus 

Find the tail number of the top ten planes (identified by `tailNum`), with the worst departure delays (average delays).  You may find `drop_duplicates`, `agg`, and `sort_values` helpful. 

Consider only planes that have made at least 10 flights. 

Report out results in a DataFrame, `topDelayed`, the tail number `tailNum`, number of flights `num`, and the mean delay `mnDelay`. 

In [ ]:
# Find the tail numbers of the planes with the worst mean dep. delays. 
# Store the tail number "tailNum", number of flights "num", and
#   mean delay "mnDelay" in a DataFrame


topDelayed = ...
...

topDelayed.head(10)

In [ ]:
grader.check("b1")

## Congratulations! You have finished P2! 

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Once you submit this file to the P2 assignment on Gradescope, Gradescope will automatically submit a PDF file with your some of your answers to the P2 - Figures assignment (making them easier to grade). 

**Important**: Please check that your responses were generated and submitted correctly to the P2 - Figures Assignment. 

**You are responsible for ensuring your submission follows our requirements and that the PDF for P2- Figures answers was generated/submitted correctly. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with the submission, contact course staff well-ahead of the deadline. 

Make sure you have run all cells in your notebook **in order** before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)